# High-Resolution Deep Convolutional Generative Adversarial Networks
J. D. Curtó, I. C. Zarza, Fernando de la Torre, Irwin King, Michael R. Lyu

## Data Preparation

In [ ]:
from crawler import crawler, purify_sources

# load urls of images into local file
crawler.run()
purify_sources.purify()


In [ ]:
from crawler import downloader

# download actual images into folder
downloader.run()

In [ ]:
from preprocessing import cropper

cropper.crop_to_face()

## Tensorflow Setup

In [ ]:
import tensorflow as tf

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Data Preparation

In [ ]:
# MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_train = (x_train - 255/2.) / (255/2.) # Normalize the images to [-1, 1]
x_train.shape

## Training

In [ ]:
from gan import GAN, DCGAN

gan = DCGAN(shape=(28,28,1))

gan.set_training_data(x_train)

In [ ]:
gan.train(epochs=50)

In [ ]:
import matplotlib.pyplot as plt

img = gan.generate()
plt.imshow((img[0, :,:,0] ), cmap='gray')
plt.show()

In [ ]:
gan.export()

In [ ]:
gan.import_()

In [ ]:
import numpy as np
noise = np.random.normal(0, 1, (5, 100))
batch_fake = gan.generator.predict(noise)

res = gan.discriminator.predict(batch_fake)
res

In [ ]:
res = gan.discriminator.predict(x_train[15:20])
res

## Create GIF

In [ ]:
gan.generate_gif()

In [ ]:
import imageio
import glob

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('gan/models/dcgan explicit/images/image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
# pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.vis.embed as embed 

embed.embed_file(anim_file)